In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
from os import *
import random

from sklearn.utils import shuffle
import tensorflow as tf

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from keras.layers import Dense, Flatten, Dropout, Activation, BatchNormalization, Conv2D, MaxPooling2D ,ZeroPadding2D, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.optimizers import SGD , Adam , Adadelta
from keras import backend as K
from keras import regularizers
from keras.regularizers import *
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from keras.callbacks import *


In [3]:
train = '../input/accident-detection-from-cctv-footage/data/train/'
val = '../input/accident-detection-from-cctv-footage/data/val/'


In [4]:
img_width,img_height = 300,300

training_size = 600
epochs = 50
batch_size = 32

if K.image_data_format()=='channels_first':
    input_shape = (3,img_width,img_height)
else:
    input_shape = (img_width,img_height,3)


In [5]:
train_datagen = ImageDataGenerator(
                rescale = 1./255.,
            
)

train_gen = train_datagen.flow_from_directory(
            train,
            target_size = (img_width,img_height),
            batch_size = batch_size,
            color_mode = 'rgb',
            class_mode = 'binary'
)

val_datagen =  ImageDataGenerator(
                rescale = 1./255.
)
validation_gen = val_datagen.flow_from_directory(
                 val,
                 target_size = (img_width,img_height),
                 batch_size = batch_size,
                 color_mode = 'rgb',
                 class_mode = 'binary')


Found 791 images belonging to 2 classes.
Found 98 images belonging to 2 classes.


In [6]:
train_gen.class_indices


{'Accident': 0, 'Non Accident': 1}

In [7]:
model = Sequential()

model.add(Conv2D (32,(3,3),input_shape=(300,300,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D (64,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D (128,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D (256,(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [8]:
opt = Adam(lr = 0.001)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])


In [9]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 activation (Activation)     (None, 298, 298, 32)      0         
                                                                 
 batch_normalization (Batch  (None, 298, 298, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 149, 149, 32)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 149, 149, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      1

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch = 24,
                    validation_data = validation_gen ,
                    validation_steps = 6,
                    epochs = 50,
                    verbose = 1)

model.save('accident_detection_model.h5')


Epoch 1/50
24/24 [==============================] - 136s 5s/step - loss: 0.9532 - accuracy: 0.6034 - val_loss: 1.5657 - val_accuracy: 0.6224
Epoch 2/50
24/24 [==============================] - 122s 5s/step - loss: 0.5761 - accuracy: 0.7009
Epoch 3/50
24/24 [==============================] - 122s 5s/step - loss: 0.5260 - accuracy: 0.7233
Epoch 4/50
24/24 [==============================] - 121s 5s/step - loss: 0.4556 - accuracy: 0.7813
Epoch 5/50
24/24 [==============================] - 123s 5s/step - loss: 0.3694 - accuracy: 0.8406
Epoch 6/50
24/24 [==============================] - 123s 5s/step - loss: 0.2989 - accuracy: 0.8709
Epoch 7/50
24/24 [==============================] - 123s 5s/step - loss: 0.2448 - accuracy: 0.9065
Epoch 8/50
24/24 [==============================] - 122s 5s/step - loss: 0.2976 - accuracy: 0.8709
Epoch 9/50
24/24 [==============================] - 122s 5s/step - loss: 0.2392 - accuracy: 0.8986
Epoch 10/50
24/24 [==============================] - 122s 5s/step -